<a href="https://colab.research.google.com/github/pdomins/bayesian-learning/blob/master/ej3_bayesian_net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [81]:
import pandas as pd

NameError: ignored

In [82]:
df = pd.read_csv("3-binary(TP1).csv", delimiter=',', encoding='utf-8')
df

,admit,gre,gpa,rank
0,0,380,3.61,3
1,1,660,3.67,3
2,1,800,4.00,1
3,1,640,3.19,4
4,0,520,2.93,4
...,...,...,...,...
395,0,620,4.00,2
396,0,560,3.04,3
397,0,460,2.63,2
398,0,700,3.65,2


In [83]:
# Discretize GRE and GPA columns
df['gre_d'] = pd.cut(df['gre'], bins=[-float('inf'), 500, float('inf')], labels=['GRE < 500', 'GRE ≥ 500'])
df['gpa_d'] = pd.cut(df['gpa'], bins=[-float('inf'), 3, float('inf')], labels=['GPA < 3', 'GPA ≥ 3'])

print(df)

     admit  gre   gpa  rank      gre_d    gpa_d
0        0  380  3.61     3  GRE < 500  GPA ≥ 3
1        1  660  3.67     3  GRE ≥ 500  GPA ≥ 3
2        1  800  4.00     1  GRE ≥ 500  GPA ≥ 3
3        1  640  3.19     4  GRE ≥ 500  GPA ≥ 3
4        0  520  2.93     4  GRE ≥ 500  GPA < 3
..     ...  ...   ...   ...        ...      ...
395      0  620  4.00     2  GRE ≥ 500  GPA ≥ 3
396      0  560  3.04     3  GRE ≥ 500  GPA ≥ 3
397      0  460  2.63     2  GRE < 500  GPA < 3
398      0  700  3.65     2  GRE ≥ 500  GPA ≥ 3
399      0  600  3.89     3  GRE ≥ 500  GPA ≥ 3

[400 rows x 6 columns]


In [84]:
# Calculate probabilities
total_count = len(df)
prob_admit_0 = df['admit'].value_counts(normalize=True)[0]
prob_admit_1 = df['admit'].value_counts(normalize=True)[1]

In [93]:
#sin Laplace
# conditional_probs = df.groupby(['gre_d', 'gpa_d', 'rank'])['admit'].value_counts(normalize=True).unstack().fillna(0)

#con Laplace
conditional_probs = df.groupby(['gre_d', 'gpa_d', 'rank'])['admit'].value_counts().unstack().fillna(0)

for index, row in conditional_probs.iterrows():
    accepted = row[1]
    not_accepted = row[0]
    total = accepted+not_accepted
    k = 2
    row[0] = (not_accepted + 1)/(total + k)
    row[1] = (accepted + 1)/(total + k)

conditional_probs

admit                          0         1
gre_d     gpa_d   rank                    
GRE < 500 GPA < 3 1     0.714286  0.285714
                  2     0.545455  0.454545
                  3     0.928571  0.071429
                  4     0.900000  0.100000
          GPA ≥ 3 1     0.500000  0.500000
                  2     0.791667  0.208333
                  3     0.791667  0.208333
                  4     0.823529  0.176471
GRE ≥ 500 GPA < 3 1     0.200000  0.800000
                  2     0.761905  0.238095
                  3     0.545455  0.454545
                  4     0.625000  0.375000
          GPA ≥ 3 1     0.448980  0.551020
                  2     0.582524  0.417476
                  3     0.737500  0.262500
                  4     0.775000  0.225000

## a) Calcular la probabilidad de que una persona que proviene de una escuela con rango 1 no haya sido admitida en la universidad

In [86]:
# rank 1
conditional_probs_rank_1 = conditional_probs[conditional_probs.index.get_level_values('rank') == 1]
conditional_probs_rank_1

admit                          0         1
gre_d     gpa_d   rank                    
GRE < 500 GPA < 3 1     0.714286  0.285714
          GPA ≥ 3 1     0.500000  0.500000
GRE ≥ 500 GPA < 3 1     0.200000  0.800000
          GPA ≥ 3 1     0.448980  0.551020

In [87]:
# P(admit = 0 | rank = 1) = P(admit = 0, rank = 1) / P(rank = 1)
#                         = sum(P(admit = 0, gre_d, gpa_d, rank = 1)) / sum(P(rank = 1, gre_d, gpa_d, admit))
#                           first column of conditional_probs_rank_1 / all values of conditional_probs_rank_1

probability = conditional_probs_rank_1.iloc[:, 0].sum() / conditional_probs_rank_1.sum().sum()
print(f'P(admit = 0 | rank = 1): {probability}')

P(admit = 0 | rank = 1): 0.4658163265306123


## b) Calcular la probabilidad de que una persona que fue a una escuela de rango 2, tenga GRE = 450 y GPA = 3.5 sea admitida en la universidad

In [88]:
# rank 2
conditional_probs_rank_2 = conditional_probs[conditional_probs.index.get_level_values('rank') == 2]
conditional_probs_rank_2

admit                          0         1
gre_d     gpa_d   rank                    
GRE < 500 GPA < 3 2     0.545455  0.454545
          GPA ≥ 3 2     0.791667  0.208333
GRE ≥ 500 GPA < 3 2     0.761905  0.238095
          GPA ≥ 3 2     0.582524  0.417476

In [89]:
# P(admit = 1, gre_d = 450 , gpa_d = 3.5| rank = 2) => P(admit = 1, gre_d < 500 , gpa_d ≥ 3 | rank = 2)
#                                                    = P(admit = 1, gre_d < 500 , gpa_d ≥ 3) / P (rank = 2)
#                                                    = sum(P(admit = 1, gre_d < 500 , gpa_d ≥ 3, rank = 2)) / sum(P (rank = 2, admit, gre_d, gpa_d))

probability =  conditional_probs_rank_2.loc[('GRE < 500', 'GPA ≥ 3', 2), 1] / conditional_probs_rank_2.sum().sum()
print(f'P(admit = 1, gre_d < 500 , gpa_d ≥ 3 | rank = 2): {probability}')

P(admit = 1, gre_d < 500 , gpa_d ≥ 3 | rank = 2): 0.05208333333333334


### ❌ ❌ ❌ ❌ ❌

In [90]:
# P(admit = 1| rank = 2, GRE = 450 , GPA = 3.5) => P(admit = 1 | rank = 2, GRE < 500 , GPA ≥ 3)
probability = conditional_probs_rank_2.loc[('GRE < 500', 'GPA ≥ 3', 2), 1]
print(f'P(admit = 1 | rank = 2, GRE < 500 , GPA ≥ 3): {probability}')

P(admit = 1 | rank = 2, GRE < 500 , GPA ≥ 3): 0.20833333333333334
